In [2]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters import * 

In [3]:
data = io.load('data/gauss4')
# data = io.load('data/simulations')

In [4]:
import warnings

warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [5]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

### Helper functions

#### view

In [6]:
def update_view(view, f):
    def _inner():
        view.show = filter_tree(view.tree, f)
    return _inner 

In [7]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [8]:
def reduce_view(widget, tree, f):
    @debug.capture()
    def _inner():
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [9]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [10]:
tw = TreeWidget(data.tree)

In [11]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [12]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

In [13]:
v2

TreeView(field='fitness', show={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, …

In [14]:
f2

### Reduce tree

In [15]:
v2, r2, p2 = reduce(tw, attr='span', title='reduce')

In [16]:
v2.attr = 'parent_fitness'

### fitness

#### test

In [17]:
tw2 = TreeWidget(data.tree)

In [18]:
f = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [19]:
def test():
    tw2.ref = reduce_tree(tw.tree, filter=f)

In [20]:
m = Monitor(f, func=test)

In [21]:
test_view = TreeView(tw2, attr='span') 

In [22]:
p = SidePanel(title='Reduce initial tree')
with p:
    display(test_view, f)

In [23]:
fitness = view(tw2, attr='fitness', title='fitness')

In [24]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

### Details

In [25]:
dw = DataWidget(data=data)

In [26]:
details = DetailsView(data=dw)

In [27]:
p2 = SidePanel(title='details')
with p2:
    display(details)

In [28]:
l1 = widgets.link((fitness[0], 'details'), (details, 'show'))

In [29]:
l2 = widgets.link((parent_fitness[0], 'details'), (details, 'show'))

In [30]:
parent_fitness[0].details = []

In [31]:
fitness[0].details = []